## StructuredOutputParser
**간단한 구조화된 출력**을 위한 파서로, 스키마를 딕셔너리 형태로 정의합니다.

**주요 특징:**

- **간단한 설정**: ResponseSchema로 쉽게 구조 정의
- **딕셔너리 기반 스키마**: 복잡한 클래스 정의 없이 간단히 사용
- **빠른 프로토타이핑**: 초기 개발 단계에서 빠르게 테스트 가능
- **유연한 필드 정의**: 설명과 함께 필드 정의

**활용 사례:**

- 제품 리뷰 분석 (평점, 장점, 단점 등)
- 간단한 설문 응답 구조화
- 빠른 프로토타입 개발
- 단순한 정보 추출 작업

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

In [2]:
# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="rating", description="5점 만점에서 예상 평점"),
    ResponseSchema(name="pros", description="리뷰에서 언급된 장점 3가지를 리스트로 출력"),
    ResponseSchema(name="cons", description="리뷰에서 언급된 단점 3가지를 리스트로 출력"),
    ResponseSchema(name="summary", description="리뷰를 한 문장으로 요약")
]

In [3]:
# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
template = """
다음 제품 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 테스트 리뷰 데이터
review = """
이 스마트폰은 배터리 수명이 정말 좋아서 하루 종일 사용해도 충전이 필요 없었어요. 
카메라 화질도 선명하고, 특히 야간 모드가 훌륭합니다. 
다만 가격이 조금 비싸고, 무게가 200g이 넘어서 손이 피곤할 수 있어요.
"""

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rating": string  // 5점 만점에서 예상 평점
	"pros": string  // 리뷰에서 언급된 장점 3가지를 리스트로 출력
	"cons": string  // 리뷰에서 언급된 단점 3가지를 리스트로 출력
	"summary": string  // 리뷰를 한 문장으로 요약
}
```
===== 분석 결과 =====
{'cons': ['가격이 비쌈', '무게가 200g 이상임', '손이 피곤할 수 있음'],
 'pros': ['배터리 수명이 좋음', '카메라 화질이 선명함', '야간 모드가 훌륭함'],
 'rating': '4',
 'summary': '스마트폰의 배터리 수명과 카메라 화질은 좋지만, 가격이 비싸고 무겁다는 리뷰입니다.'}
